# Investigating the USV data from Paper 2

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#file_name = "zigzag_30_inter"
file_name = "turnningdata"

data = pd.read_csv(f"{file_name}.csv")
#data['time']/=10
data.set_index('time', inplace=True)
data.sort_index(inplace=True)

if file_name == "turnningdata":
    data = data.iloc[0:190]

data.index = pd.to_datetime(data.index, unit='s')
data['rudder']-=40
data['delta'] = -np.deg2rad(data['rudder'])
data['psi'] = np.unwrap(np.deg2rad(data['heading']))
data['thrust'] = 0
data = data.resample('0.5S').mean().dropna()

data.index = (data.index-data.index[0]).total_seconds()




In [ ]:
data['psi'].diff() > 0.8*np.pi

In [ ]:
data.head()

In [ ]:
data['psi'].diff().abs().max()

In [ ]:
data.describe()

In [ ]:
data.index[-1]

In [ ]:
fig,ax=plt.subplots()
ax.plot(np.diff(data.index))

In [ ]:
np.mean(np.diff(data.index))

In [ ]:
data.dtypes

In [ ]:
data['-rudder'] = -data['rudder']
data.plot(y=['psi','delta'], style='.-')

In [ ]:
data

In [ ]:
df = geopandas.GeoDataFrame(data.copy(), 
                 geometry=geopandas.points_from_xy(data.lon, data.lat, crs="EPSG:4326"))
df = df.to_crs(epsg=3829)
df['y0'] = df.geometry.x - df.geometry.x[0]
df['x0'] = df.geometry.y - df.geometry.y[0]
df['distance'] = df.distance(df.shift())

In [ ]:
df['distance'].sum()

In [ ]:
df.plot()

In [ ]:
df.head()

In [ ]:
fig,ax=plt.subplots()
df.plot(x='y0',y='x0', kind='line', ax=ax)
ax.axis('equal')


In [ ]:
data['x0'] = df['x0'].copy()
data['y0'] = df['y0'].copy()

In [ ]:
dt_min = np.diff(data.index).min()

In [ ]:
np.diff(data.index).max()

In [ ]:
dt_min

In [ ]:
np.round(dt_min,4)

In [ ]:
np.floor(dt_min*10**4)/(10**4)

In [ ]:
f'{dt_min_floor}S'

In [ ]:
f'{dt_min_floor}S'

In [ ]:
#data.to_csv('zigzag_30_inter_xy.csv')
data.to_csv(f'{file_name}_xy.csv')

In [ ]:
data.head()

In [ ]:
data.index[-1]